### [Capstone](http://www.capstone-engine.org/)
> Capstone is a lightweight multi-platform, multi-architecture disassembly framework.
Our target is to make Capstone the ultimate disassembly engine for binary analysis and reversing in the security community.

* [Installation](http://www.capstone-engine.org/documentation.html)
* [Python Tutorial](http://www.capstone-engine.org/lang_python.html)

In [2]:
import capstone

In [6]:
# 부팅가능하지 않은 플로피 디스크 VBR 영역 코드
code = b'\x0e\x1f\xbe[|\xac"\xc0t\x0bV\xb4\x0e\xbb\x07\x00\xcd\x10^\xeb\xf02\xe4\xcd\x16\xcd\x19\xeb\xfe'

md = capstone.Cs(capstone.CS_ARCH_X86, capstone.CS_MODE_16)
for i in md.disasm(code, 0x7c00):
    print("0x{:x}:\t{:12}\t{:s}\t{:s}".format(i.address, bytes(i.bytes).hex(), i.mnemonic, i.op_str))

0x7c00:	0e          	push	cs
0x7c01:	1f          	pop	ds
0x7c02:	be5b7c      	mov	si, 0x7c5b
0x7c05:	ac          	lodsb	al, byte ptr [si]
0x7c06:	22c0        	and	al, al
0x7c08:	740b        	je	0x7c15
0x7c0a:	56          	push	si
0x7c0b:	b40e        	mov	ah, 0xe
0x7c0d:	bb0700      	mov	bx, 7
0x7c10:	cd10        	int	0x10
0x7c12:	5e          	pop	si
0x7c13:	ebf0        	jmp	0x7c05
0x7c15:	32e4        	xor	ah, ah
0x7c17:	cd16        	int	0x16
0x7c19:	cd19        	int	0x19
0x7c1b:	ebfe        	jmp	0x7c1b


* 8bit General Purpose Register: al, bl, cl, dl
* 16bit Segment Register: cs, ds, es, fs, gs, ss
* 16bit Index Register: si, di
* lodsb: Load byte at address DS:(E)SI into AL

```
//cs에 있는 값을 ds로 옮긴다.
0x7c3e:	0e          	push	cs              
0x7c3f:	1f          	pop	ds

//문자열의 시작주소를 si에 넣는다.
0x7c40:	be5b7c      	mov	si, 0x7c5b          

//ds:si에 있는 문자열을 al로 읽어온다.
0x7c43:	ac          	lodsb	al, byte ptr [si]

//읽어온 문자가 null이면 0x7c53으로 점프한다.
0x7c44:	22c0        	and	al, al
0x7c46:	740b        	je	0x7c53

//읽어온 문자가 null이 아니면 si에 있는 현재 문자열 주소를 스택에 저장한다.
0x7c48:	56          	push	si

//읽어온 문자를 화면에 출력한다.
0x7c49:	b40e        	mov	ah, 0xe
0x7c4b:	bb0700      	mov	bx, 7
0x7c4e:	cd10        	int	0x10

//다시 스택에 저장된 문자열 주소를 꺼내와 si에 넣고 0x7c43으로 점프한다. 
0x7c50:	5e          	pop	si
0x7c51:	ebf0        	jmp	0x7c43

//키보드로부터 문자를 읽어온다.
0x7c53:	32e4        	xor	ah, ah
0x7c55:	cd16        	int	0x16

//재부팅한다.
0x7c57:	cd19        	int	0x19

//아무것도 하지 않는 무한루프에 들어간다.
0x7c59:	ebfe        	jmp	0x7c59

//문자열
0x7c5b: 54686973206973206e6f74206120626f6f7461626c65206469736b2e2020506c6561736520696e73657274206120626f6f7461626c6520666c6f70707920616e640d0a707265737320616e79206b657920746f2074727920616761696e202e2e2e200d0a00
```